<div class="alert alert-block alert-info">
<h1 style="text-align:center;font-family:avenir;">ICA first</h1>
Lou Planchamp & Victor Férat
</div>

In [ ]:
%matplotlib inline

In [ ]:
from nilearn import plotting
from nilearn.plotting import plot_stat_map
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
from numpy import array
import os
from nilearn.plotting import plot_design_matrix
import numpy as np
import matplotlib.pyplot as plt
from nilearn.glm.second_level import SecondLevelModel

from os.path import join
from bids import BIDSLayout
from bids.tests import get_test_data_path
from nilearn.plotting import plot_prob_atlas
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show


from nilearn.decomposition import CanICA

Here, thanks to BIDSLayout, we define whitch files we want to work on (all the resting states) and whitch files to use for the preprocessing (confounds)

In [ ]:
bids_folder = '/home/carole/Bureau/BIDS_fMRIprep'
layout = BIDSLayout(bids_folder, derivatives=True)
layout

In [ ]:
func_filenames = layout.get(subject=['01','02','08'], task='rest', run=[1,2], space='MNIPediatricAsym', desc='preproc', extension='nii.gz', return_type='filename')

In [ ]:
func_filenames

In [ ]:
confounds = layout.get(task='rest',  run=[1,2], desc='confounds', extension='tsv', return_type='filename')

In [ ]:
confounds

Group ICA

In [ ]:
canica = CanICA(n_components=20,
                memory="nilearn_cache", memory_level=2,
                verbose=10,
                mask_strategy='whole-brain-template',
                random_state=0)
canica.fit(func_filenames)

# Retrieve the independent components in brain space. Directly accessible through attribute `components_img_`.
canica_components_img = canica.components_img_

# components_img is a Nifti Image object, and can be saved to a file with the following line:

if not os.path.isdir('ICA'): 
    os.mkdir('ICA')
if not os.path.isdir('ICA/group'): 
    os.mkdir('ICA/group')
canica_components_img.to_filename('ICA/group/canica_resting_state_wo-sub.nii.gz')

In [ ]:
plot_prob_atlas(canica_components_img, title='All ICA components', output_file='ICA/group/canica_resting_state_wo-sub.png')

In [ ]:
n_columns = 2
n_ica = 20
nrows = int(n_ica / n_columns)
fig, axes = plt.subplots(nrows=nrows, ncols=n_columns, figsize=(20,60))

for i, cur_img in enumerate(iter_img(canica_components_img)):
    plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                  cut_coords=3, 
                  colorbar=False, 
                  axes=axes[int(i / 2), int(i % 2)],
                  output_file='ICA/group/canica_resting_state_wo-sub_fig2.png')

This following loop does an ICA for each subject individualy

In [ ]:
for subject in layout.get_subjects():
    if subject not in ['07']:
        for session in layout.get_sessions(subject=subject):
            f = layout.get(task='rest', subject=['07'], session=session,
                           space='MNIPediatricAsym', desc='preproc', extension='nii.gz', 
                           return_type='filename')

            print(subject, session, f)

            canica = CanICA(n_components=20,
                            memory="nilearn_cache", memory_level=2,
                            verbose=1,
                            mask_strategy='whole-brain-template',
                            smoothing_fwhm=10,
                            random_state=0)
            canica.fit(f)


            if not os.path.isdir('ICA'):
                os.mkdir('ICA')
            if not os.path.isdir('ICA/individual'): 
                os.mkdir('ICA/individual')
            sub_folder = f'ICA/individual/{subject}'
            if not os.path.isdir(sub_folder): 
                os.mkdir(sub_folder)

            sub_path = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_ica.nii')

            # Retrieve the independent components in brain space. Directly ccessible through attribute `components_img_`.
            canica_components_img = canica.components_img_
            # components_img is a Nifti Image object, and can be saved to a file with he following line:
            canica_components_img.to_filename(sub_path)
            
            sub_path_fig1 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig1_ica.png')

            plot_prob_atlas(canica_components_img, output_file=sub_path_fig1)
            
            n_columns = 2
            n_ica = 20
            nrows = int(n_ica / n_columns)
            fig, axes = plt.subplots(nrows=nrows, ncols=n_columns, figsize=(20,60))
            
            sub_path_fig2 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig2_ica.png')

            for i, cur_img in enumerate(iter_img(canica_components_img)):
                plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                              output_file=sub_path_fig2,
                              cut_coords=3, 
                              colorbar=False, 
                              axes=axes[int(i / 2), int(i % 2)])